In [1]:
!pip install captum

     |████████████████████████████████| 4.4MB 22.1MB/s 


In [2]:
import captum

import spacy

import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F
from torchtext.legacy.data import BucketIterator
from torchtext.vocab import GloVe
from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.datasets import SST
import torch.optim as optim


from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization, LayerDeepLift, LayerConductance, LayerFeatureAblation, DeepLift, Deconvolution

nlp = spacy.load('en')

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
def preprocess_labels(score):
  print(score)
  if score >= 0 and score < 0.2:
      label = 0
  elif score >= 0.2 and score < 0.4:
      label = 1
  elif score >= 0.4 and score < 0.6:
      label = 2
  elif score >=0.6 and score < 0.8:
      label = 3
  else:
      label = 4
  return label 

In [5]:
def get_sst_data(device):
    # set up fields
    #TEXT = Field(lower=True, include_lengths=True, batch_first=True)
    #LABEL =Field(sequential=False)
    TEXT = Field(lower=True, tokenize='spacy', batch_first=True)
    Label = LabelField(dtype = torch.float, use_vocab=True, preprocessing=float)


    # make splits for data
    train, val, test = SST.splits(TEXT, Label, fine_grained=True)

    # build the vocabulary
    loaded_vectors = GloVe(name='6B', dim=100)
    loaded_vectors = torchtext.vocab.Vectors('glove.6B.100d.txt')
    TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
    TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)

    Label.build_vocab(train)
    text_vocab = TEXT.vocab
    x = torch.tensor(1)

    # make iterator for splits
    train_iter, val_iter, test_iter = BucketIterator.splits(
        (train, val, test), batch_size=32, device=torch.device('cuda:0')) # for CPU, device = -1

    return train_iter, val_iter, test_iter, text_vocab, TEXT, Label

In [6]:
train_iter, val_iter, test_iter, vocab, TEXT, Label = get_sst_data(device)

downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 935kB/s]


extracting


.vector_cache/glove.6B.zip: 862MB [02:41, 5.33MB/s]                           
100%|█████████▉| 399289/400000 [00:14<00:00, 26868.42it/s]

In [7]:
class TextCNN(nn.Module):
    def __init__(self, vocab,  embedding_dim, n_filters, num_classes, dropout, pad_idx = 0, window_sizes=(1,2,3,5)):
        super(TextCNN, self).__init__()
        # load pretrained embedding in embedding layer.
        self.embedding = nn.Embedding(len(vocab), embedding_dim, padding_idx = pad_idx)
        #self.embedding.weight.data.copy_(vocab.vectors)

        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(window_size, embedding_dim), padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.fc = nn.Linear(n_filters * len(window_sizes), num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, max_sent_len=49):
        #text_emb = text_emb.unsqueeze(1).unsqueeze(2)
        #text_emb = text_emb.expand(-1,1,max_sent_len,-1).to(device)

        #text = [batch size, sent len]
        
        text_emb = self.embedding(text)

        # text_emb = [batch size, sent len, emb dim]

        text_emb = text_emb.unsqueeze(1)

        # text_emb = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(text_emb)).squeeze(3) for conv in self.convs]
        # conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))
        # cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)


In [8]:
model = TextCNN(vocab, embedding_dim=100, n_filters=100, num_classes=5, dropout=0.5, window_sizes=(3,), pad_idx=0)


In [9]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

In [12]:
for key in Label.vocab.freqs:
  print(key, Label.vocab.stoi[key])

positive 0
very positive 3
neutral 2
negative 1
very negative 4


In [13]:
def train(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0

    model.train() # sets the training mode

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text)
        true_labels = batch.label.long()
      
        #true_labels = torch.sub(batch.label, torch.tensor(1))
        #true_labels = true_labels.to(device)

        # Calc loss
        loss = criterion(predictions, true_labels)
        acc = categorical_accuracy(predictions, true_labels)

        # Calcualte train and validation losses after 10 episodes
        # Backprop step
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc/len(iterator), model


def evaluate(model, iterator, criterion, device, split='val'):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()
    #num_iterations = 0
    with torch.no_grad():
        #for emb, labels in get_batched_emb_labels(args, split=split):
        for batch in iterator:
            predictions = model(batch.text)
            true_labels = batch.label.long()
            
            #true_labels = torch.sub(batch.label, torch.tensor(1))
            
            #true_labels = true_labels.to(device)

            loss = criterion(predictions, true_labels)
            acc = categorical_accuracy(predictions, true_labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            #num_iterations += 1

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
print('Vocabulary Size: ', len(TEXT.vocab))


Vocabulary Size:  15480


In [15]:
PAD_IND = TEXT.vocab.stoi['pad']


In [16]:
PAD_IND

12760

In [17]:
def categorical_accuracy(preds, y):
    """
    We calculate the accuracy by performing an argmax to get the index of the maximum value in the prediction for
    each element in the batch, and then counting how many times this equals the actual label.
    We then average this across the batch.
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [18]:
model = model.to("cuda:0")
optimizer = optim.Adam(model.parameters(), lr=float(1e-4))
loss_fn = nn.CrossEntropyLoss()  # nn.NLLLoss()
# CrossEntropyLoss expects the input to be [batch size, n classes] and the label to be [batch size].
loss_fn.to(device)

#SST.iters(batch_size=32, device=None) # change device to -1 if using cpu

for epoch in range(50):

    train_loss, train_acc, model = train(model, train_iter, optimizer, loss_fn, device)
    # calculate validation loss and accuracy
    valid_loss, valid_acc = evaluate(model, val_iter, loss_fn, device, split='val')

    print(f'Epoch: {epoch + 1} ')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')
torch.save(model.state_dict(), f"cnn_model_50.pt")

test_loss, test_acc = evaluate(model, test_iter, loss_fn, device, split='test')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


100%|█████████▉| 399289/400000 [00:29<00:00, 26868.42it/s]

Epoch: 1 
	Train Loss: 1.711 | Train Acc: 22.67%
	 Val. Loss: 1.567 |  Val. Acc: 29.32%
Epoch: 2 
	Train Loss: 1.642 | Train Acc: 25.50%
	 Val. Loss: 1.558 |  Val. Acc: 30.57%
Epoch: 3 
	Train Loss: 1.621 | Train Acc: 26.45%
	 Val. Loss: 1.552 |  Val. Acc: 31.46%
Epoch: 4 
	Train Loss: 1.589 | Train Acc: 27.81%
	 Val. Loss: 1.547 |  Val. Acc: 32.66%
Epoch: 5 
	Train Loss: 1.561 | Train Acc: 29.81%
	 Val. Loss: 1.543 |  Val. Acc: 31.82%
Epoch: 6 
	Train Loss: 1.551 | Train Acc: 30.37%
	 Val. Loss: 1.537 |  Val. Acc: 32.18%
Epoch: 7 
	Train Loss: 1.543 | Train Acc: 30.61%
	 Val. Loss: 1.535 |  Val. Acc: 32.53%
Epoch: 8 
	Train Loss: 1.535 | Train Acc: 31.17%
	 Val. Loss: 1.528 |  Val. Acc: 32.98%
Epoch: 9 
	Train Loss: 1.512 | Train Acc: 32.90%
	 Val. Loss: 1.525 |  Val. Acc: 33.07%
Epoch: 10 
	Train Loss: 1.503 | Train Acc: 34.02%
	 Val. Loss: 1.522 |  Val. Acc: 33.16%
Epoch: 11 
	Train Loss: 1.495 | Train Acc: 34.42%
	 Val. Loss: 1.517 |  Val. Acc: 33.78%
Epoch: 12 
	Train Loss: 1.488 

In [19]:
torch.load('cnn_model_50.pt')
model.eval()

TextCNN(
  (embedding): Embedding(15480, 100, padding_idx=0)
  (convs): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1), padding=(2, 0))
  )
  (fc): Linear(in_features=100, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [20]:
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)


In [21]:
lig = LayerIntegratedGradients(model, model.embedding)
#lig = LayerIntegratedGradients(model, model.convs)
dl = DeepLift(model)

In [22]:
# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 30, true_label = 0):
    text = [tok.text for tok in nlp.tokenizer(sentence.lower())]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)

    #print(input_indices.shape)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices) #.item()
    print(pred)
    score = torch.max(pred).item()
    print(score)
    if score >= 0 and score < 0.2:
      label = Label.vocab.stoi["very negative"]
    elif score >= 0.2 and score < 0.4:
      label = Label.vocab.stoi["negative"]
    elif score >= 0.4 and score < 0.6:
      label = Label.vocab.stoi["neutral"]
    elif score >=0.6 and score < 0.8:
      label = Label.vocab.stoi["positive"]
    else:
      label = Label.vocab.stoi["very positive"]
    pred_ind = label #round(pred)
    #print(label)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
    #print(reference_indices)
    #print(input_indices)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, target=pred_ind, \
                                           n_steps=500, return_convergence_delta=True)
    
    #print(Label.vocab.itos[pred_ind])

    print(f"pred: {Label.vocab.itos[pred_ind]}, {label}, delta: {abs(delta)}")

    add_attributions_to_visualizer(attributions_ig, text, label, pred_ind, true_label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            Label.vocab.itos[pred_ind],
                            Label.vocab.itos[label],
                            Label.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [23]:
for key in Label.vocab.freqs:
  print(key, Label.vocab.stoi[key])

positive 0
very positive 3
neutral 2
negative 1
very negative 4


In [24]:
interpret_sentence(model, 'It was a fantastic performance !', true_label=3)
interpret_sentence(model, "Renner 's performance as Dahmer is unforgettable , deeply absorbing .", true_label=3)
interpret_sentence(model, 'Best film ever', true_label=3)
interpret_sentence(model, 'Too leisurely paced and visually drab for its own good , it succeeds in being only sporadically amusing .', true_label=2)
interpret_sentence(model, 'It was a horrible movie', true_label=1)
interpret_sentence(model, 'I\'ve never watched something as bad', true_label=4)
interpret_sentence(model, 'It is a disgusting movie!', true_label=4)

tensor([[0.6607, 0.2069, 0.5774, 0.7581, 0.1603]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
0.7580726146697998
pred: positive, 0, delta: tensor([0.0001], device='cuda:0', dtype=torch.float64)
tensor([[0.8077, 0.1877, 0.3625, 0.7534, 0.1246]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
0.8077468872070312
pred: very positive, 3, delta: tensor([5.6122e-05], device='cuda:0', dtype=torch.float64)
tensor([[0.6184, 0.1784, 0.3403, 0.9118, 0.1656]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
0.911780059337616
pred: very positive, 3, delta: tensor([0.0013], device='cuda:0', dtype=torch.float64)
tensor([[0.4955, 0.5837, 0.4865, 0.3659, 0.3169]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
0.5837150812149048
pred: neutral, 2, delta: tensor([0.0004], device='cuda:0', dtype=torch.float64)
tensor([[0.5237, 0.3832, 0.3331, 0.7205, 0.4246]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
0.7205315828323364
pred: positive, 0, delta: tensor([0.0004], device='cuda:0',

In [25]:
print('Visualize attributions based on Integrated Gradients')
_ = visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


Captum visualization library shows in green tokens that push the prediction towards the target class. Those driving the score towards the reference value are marked in red. As a result, words perceived as positive will appear in green if attribution is performed against the predicted class but will be highlighted in red with an attribution targeting attributed label class.

Because importance scores ar assigned to tokens, not words, some examples may show, that attribution is highly dependent on tokenization. Classification results may vary between runs.